In [1]:
import pandas as pd
import sqlite3
conn = sqlite3.connect("zippedData/im.db")

In [13]:
%%bash
sqlite3 zippedData/im.db
.schema
# just checking the .schema to get a better understanding of the data

CREATE TABLE IF NOT EXISTS "movie_basics" (
"movie_id" TEXT,
  "primary_title" TEXT,
  "original_title" TEXT,
  "start_year" INTEGER,
  "runtime_minutes" REAL,
  "genres" TEXT
);
CREATE TABLE IF NOT EXISTS "directors" (
"movie_id" TEXT,
  "person_id" TEXT
);
CREATE TABLE IF NOT EXISTS "known_for" (
"person_id" TEXT,
  "movie_id" TEXT
);
CREATE TABLE IF NOT EXISTS "movie_akas" (
"movie_id" TEXT,
  "ordering" INTEGER,
  "title" TEXT,
  "region" TEXT,
  "language" TEXT,
  "types" TEXT,
  "attributes" TEXT,
  "is_original_title" REAL
);
CREATE TABLE IF NOT EXISTS "movie_ratings" (
"movie_id" TEXT,
  "averagerating" REAL,
  "numvotes" INTEGER
);
CREATE TABLE IF NOT EXISTS "persons" (
"person_id" TEXT,
  "primary_name" TEXT,
  "birth_year" REAL,
  "death_year" REAL,
  "primary_profession" TEXT
);
CREATE TABLE IF NOT EXISTS "principals" (
"movie_id" TEXT,
  "ordering" INTEGER,
  "person_id" TEXT,
  "category" TEXT,
  "job" TEXT,
  "characters" TEXT
);
CREATE TABLE IF NOT EXISTS "writers" (
"m

In [3]:
pd.read_sql("""
SELECT *
FROM movie_akas
JOIN movie_basics
USING (movie_id)
WHERE region IN ('US', "CA")
""",conn)
# checking movies and flittering for movies in the North American region

,movie_id,ordering,title,region,language,types,attributes,is_original_title,primary_title,original_title,start_year,runtime_minutes,genres
0,tt0369610,21,Jurassic World 3D,US,None,None,3-D version,0.0,Jurassic World,Jurassic World,2015,124.0,"Action,Adventure,Sci-Fi"
1,tt0369610,28,Monde jurassique,CA,fr,imdbDisplay,None,0.0,Jurassic World,Jurassic World,2015,124.0,"Action,Adventure,Sci-Fi"
2,tt0369610,29,Jurassic World,US,None,None,None,0.0,Jurassic World,Jurassic World,2015,124.0,"Action,Adventure,Sci-Fi"
3,tt0369610,2,Ebb Tide,US,None,None,fake working title,0.0,Jurassic World,Jurassic World,2015,124.0,"Action,Adventure,Sci-Fi"
4,tt0369610,36,Jurassic Park IV,US,None,working,None,0.0,Jurassic World,Jurassic World,2015,124.0,"Action,Adventure,Sci-Fi"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
60356,tt9310460,6,Coconut the Little Dragon 2 Into the Jungle,US,None,imdbDisplay,None,0.0,Coconut the Little Dragon 2 Into the Jungle,Der kleine Drache Kokosnuss - Auf in den Dschu...,2018,NaN,"Adventure,Animation,Comedy"
60357,tt9358044,2,Hail Satan?,US,None,imdbDisplay,None,0.0,Hail Satan?,Hail Satan?,2019,95.0,Documentary
60358,tt9562694,5,Alien Warfare,US,None,imdbDisplay,None,0.0,Alien Warfare,Alien Warfare,2019,88.0,"Action,Sci-Fi"
60359,tt9705860,1,Dusan Vukotic Croatian Oscar Winner,US,None,imdbDisplay,None,0.0,Dusan Vukotic Croatian Oscar Winner,Dusan Vukotic hrvatski okarovac,2011,55.0,Documentary


In [4]:
director_df = pd.read_sql("""
SELECT movie_id, primary_name as director, primary_title, genres, start_year,averagerating,region

FROM directors
JOIN persons 
USING (person_id)

JOIN movie_basics
USING (movie_id)



JOIN movie_akas
USING (movie_id)

JOIN movie_ratings
USING (movie_id)

WHERE region IN ('US',"CA")

GROUP BY movie_id

""",conn)



# First i am joining directors and person to get the directors name
# then i am joining it with the movie_basics to get movie name, based on the id
# also making sure I have the region 
# and the ratings
#  getting the region based the north american region, because we are in the US and we share common thing with CA
# grouping by movie_id for less duplicates

In [5]:
writer_df = pd.read_sql("""
SELECT  movie_id, primary_name as writers
FROM persons 
JOIN writers
USING (person_id)

JOIN movie_basics
USING (movie_id)


""",conn)
# also want to get the writer for each movie

In [6]:
director_df.set_index('movie_id', inplace=True)

In [7]:
writer_df.set_index('movie_id', inplace=True)

In [9]:
df = director_df.join(writer_df, how='inner')
# joining the directors and writer dataset, to have both directors and writer in the same dataset
# that is why I set the index to movie_id so join will be easier

In [10]:
df.reset_index(inplace=True)
# resetting the index

In [12]:
df.head()
# taking a look at the frame
# You have to save this dataframe

,movie_id,director,primary_title,genres,start_year,averagerating,region,writers
0,tt0069049,Orson Welles,The Other Side of the Wind,Drama,2018,6.9,US,Oja Kodar
1,tt0069049,Orson Welles,The Other Side of the Wind,Drama,2018,6.9,US,Orson Welles
2,tt0137204,Anthony Harrison,Joe Finds Grace,"Adventure,Animation,Comedy",2017,8.1,CA,Anthony Harrison
3,tt0159369,John Mulholland,Cooper and Hemingway: The True Gen,Documentary,2013,7.6,US,John Mulholland
4,tt0192528,Stuart Paul,Heaven & Hell,Drama,2018,4.0,US,Stuart Paul


In [15]:
df.isna().sum()
# checking for null values

movie_id           0
director           0
primary_title      0
genres           166
start_year         0
averagerating      0
region             0
writers            0
dtype: int64

In [16]:
df.shape

(77360, 8)

In [17]:
df.dropna(subset=["genres"], inplace=True)
# because the null in genres is low, I am going to drop them based on the generes row

In [18]:
df.shape

(77194, 8)

In [26]:
dup_bool_series = df.duplicated()
dup_bool_series.sum()
# checking for duplicates in the dataframe
# I dont want duplicates

30374

In [27]:
df.drop_duplicates(inplace=True)

In [29]:
df.shape
# still have a large dataset

(46820, 8)

In [31]:
df.isna().sum()

movie_id         0
director         0
primary_title    0
genres           0
start_year       0
averagerating    0
region           0
writers          0
dtype: int64

In [33]:
df

,movie_id,director,primary_title,genres,start_year,averagerating,region,writers
0,tt0069049,Orson Welles,The Other Side of the Wind,Drama,2018,6.9,US,Oja Kodar
1,tt0069049,Orson Welles,The Other Side of the Wind,Drama,2018,6.9,US,Orson Welles
2,tt0137204,Anthony Harrison,Joe Finds Grace,"Adventure,Animation,Comedy",2017,8.1,CA,Anthony Harrison
3,tt0159369,John Mulholland,Cooper and Hemingway: The True Gen,Documentary,2013,7.6,US,John Mulholland
4,tt0192528,Stuart Paul,Heaven & Hell,Drama,2018,4.0,US,Stuart Paul
...,...,...,...,...,...,...,...,...
77355,tt9825006,Rémi St-Michel,Avant qu'on explose,Comedy,2019,6.9,CA,Eric K. Boulianne
77356,tt9845398,Lucio Castro,End of the Century,Drama,2019,7.2,US,Lucio Castro
77357,tt9852644,Loïc Darses,La Fin des terres,Documentary,2019,7.0,CA,Loïc Darses
77358,tt9866344,Adib Alkhalidey,Mon Ami Walid,"Comedy,Drama",2019,5.7,CA,Adib Alkhalidey


# Based on this dataframe, the question we can ask to solve are 

## 1. What is the best genre based on average_rating per year, in the US and CA regions, is there a trend?

(seaborn is probably best for the 1st question, x=year, y=average_rating, hue = genre)
you have to group the genere the with the average_rating for this to work

## 2. Which writer wrote the best movie based on average_rating per genres, in the US and CA regions?